In [ ]:
import os
os.environ["PYOPENGL_PLATFORM"] = "egl"
import cv2
from src.facescape_fitter import facescape_fitter
from src.utility import show_img_arr
from src.renderer import render_orthcam

fs_fitter = facescape_fitter(fs_file = "./bilinear_model_v1.6/facescape_bm_v1.6_847_50_52_id_front_new.npz", 
                             kp2d_backend = 'dlib') 
src_img = cv2.imread("./test_data/0.jpg")  
print("reading img file start")
kp2d = fs_fitter.detect_kp2d(src_img) # extract 2D key points
mesh, a = fs_fitter.fit_kp2d(kp2d) # fit model
print("program mesh ended")

# ========== Visualize Result ==========
# transform to orthogonal camera coordinate

mesh_tm = trimesh.Trimesh(vertices = mesh.vertices.copy(), 
                          faces = fs_fitter.fv_indices_front-1, 
                          process = False)
mesh_tm.vertices[:, :2] = mesh_tm.vertices[:, 0:2] - np.array([src_img.shape[1] / 2, src_img.shape[0] / 2])#原行两列
mesh_tm.vertices = mesh_tm.vertices / src_img.shape[0] * 2
mesh_tm.vertices[:, 2] = mesh_tm.vertices[:, 2] - 10

# render texture image and depth
rend_depth, rend_tex = render_orthcam(mesh_tm, (1, 1), 
                                      rend_size = tuple(src_img.shape[:2]), 
                                      flat_shading=False)
                                      
mask = np.stack((rend_depth!=0, )*3, 2)
res_img = src_img.copy()
res_img[mask] = rend_tex[mask]

print("program ended")


# visualize
vis_scale = 512. / np.max(src_img.shape[:2])
sc_img = cv2.resize(src_img, (round(src_img.shape[1]*vis_scale), 
                              round(src_img.shape[0]*vis_scale)))

sc_res_img = cv2.resize(res_img, (round(src_img.shape[1]*vis_scale), 
                                  round(src_img.shape[0]*vis_scale)))

haha = show_img_arr(np.concatenate((sc_img, sc_res_img), 1), bgr_mode = True)
print("show succeed")




